In [92]:
# Importera Pandas-bibliotek och använda "pd" som alias
import pandas as pd

In [93]:
#Läs in CSV-filen
df = pd.read_csv('./data/house_price.csv')

In [94]:
df.head()

,bedrooms,sq_meter,neighborhood,sales_price
0,3,200,Normalville,250000
1,2,80,Cool Town,300000
2,2,85,Normalville,150000
3,1,55,Normalville,78000
4,4,200,Skid Row,150000


In [95]:
df.head(3)

,bedrooms,sq_meter,neighborhood,sales_price
0,3,200,Normalville,250000
1,2,80,Cool Town,300000
2,2,85,Normalville,150000


First attempt

In [96]:
# Funktion för att uppskatta försäljningspriset
def estimate_house_sales_price(num_of_bedrooms, sqm, neighborhood):
    #Initialisera prisvariabeln
    price = 0

    # Sätta grundpris per kvadratmeter
    price_per_sqm = 200
    # Justera pris baserat på området
    if neighborhood == 'Cool Town':
        price_per_sqm = 400
    elif neighborhood == 'Skid Row':
        price_per_sqm = 100
    
    # Beräkna totalt pris
    price = price_per_sqm * sqm

    # Modifiera priset baserat på antal sovrum
    if num_of_bedrooms == 0:
        price -= 20000 
    else:
        price += num_of_bedrooms * 100000
    
    # Returnera det slutgitliga priset
    return price

In [97]:
# Anropa funktion för att uppskatta försäljningspriset
estimate_house_sales_price(4, 175, 'Skid Row')

417500

Make it more flexible

In [98]:
# Funktion för att uppskatta försäljningspriset
def estimate_house_sales_price(num_of_bedrooms, sqm, neighborhood):
    price = 0

    price += num_of_bedrooms * 0.841232951398213
    price += sqm *1231.1231231
    price += neighborhood * 2.3242341421
    price += 201.23432095
    return price

In [99]:
# for _, house in df.iterrows():
#     print(estimate_house_sales_price(house.bedrooms, house.sq_meter, house.neighborhood), house.sales_price)

In [100]:
# Skapa en dictionary för att mappa områden till numeriska värden
neighborhood_mapping = {
    'Skid Row' : 1,
    'Normalville' : 2,
    'Cool Town' : 3
}

In [101]:
# Iterera över varje rad I "df"

for _, house in df.iterrows():
    print(int(estimate_house_sales_price(house.bedrooms, house.sq_meter, neighborhood_mapping[house.neighborhood])), house.sales_price)

246433 250000
98699 300000
104853 150000
67918 78000
246431 150000


Calculate absolute error

In [102]:
# Skapa en tom lista för att lagra tuplar av beräknade priser och verkliga försäljningspriser för varje hus
predictions = []

# Iterera över varje rad #df
for _, house in df.iterrows():
# Anropa estimate_house_sales_price-funktionen med aktuella parametrar och konvertera resultatet till heltal
# Lägg sedan till en tupel av det beräknade priset och det verkliga försäljningspriset i listan
    predictions.append((int(estimate_house_sales_price(house.bedrooms, house.sq_meter, neighborhood_mapping[house.neighborhood])), house.sales_price))

# Visa den resulterande listan med tuplar
predictions

[(246433, 250000),
 (98699, 300000),
 (104853, 150000),
 (67918, 78000),
 (246431, 150000)]

In [103]:
# Skapa en tom lista för att lagra absoluta fel mellan de beräknade priserna och de verkliga försäljningspriserna 
absolute_errors = []

# Iterera över varje tupel i listan med förutsägelser
for prediction, price in predictions:
    # Beräkna det absoluta felet och lägg till det i listan
    absolute_errors.append(abs(prediction - price))
absolute_errors

[3567, 201301, 45147, 10082, 96431]

In [104]:
list_a = [1,2,3]
list_b = ['a','b','c']

list(zip(list_a, list_b))

[(1, 'a'), (2, 'b'), (3, 'c')]

In [105]:
# Funktionen för att beräkna medelvärdet av absoluta felen mellan faktiska och förutsagda värden
def mean_absolute_error(y_true, y_pread):
    # Kontrollera att längden på faktiska och förutsagda värden är samma
    assert len(y_true) == len(y_pread), 'length of actual and predicted values must be the same'

    # Beräkna absoluta fel för varje par av faktiska och förutsagda värden
    absolute_errors = [abs(true_value - pred_value) for true_value, pred_value in zip(y_true, y_pread)]

    # Beräkna och returnera medelvärdet av absoluta felen
    return sum(absolute_errors) / len(y_true)

In [106]:
# Skapa en lista med de verkliga försäljningspriserna från "df"

real_values = df.sales_price.values.tolist()
real_values

[250000, 300000, 150000, 78000, 150000]

In [107]:
# predictions = []
# for _, house in df.iterrows():
#     predictions.append((int(estimate_house_sales_price(house.bedrooms, house.sq_meter, neighborhood_mapping[house.neighborhood])), house.sales_price))
# predictions

# Skapa en lista med förutsagda försäljningspriser
predicted_values = [int(estimate_house_sales_price(house.bedrooms, house.sq_meter, neighborhood_mapping[house.neighborhood])) for _, house in df.iterrows()]
predicted_values

[246433, 98699, 104853, 67918, 246431]

In [108]:
# Beräkna medelvärdet av absoluta felen mellan de verkliga och förutsagda försäljningspriserna
mean_absolute_error(real_values, predicted_values)

71305.6

Create a weight dict to start with

In [125]:
# Skapa en dictionary med vikter för  antal sovrum osv även en konstantbias
weights_data = {
    'bedroom_weight' : 0.8,
    'sqm_weight' : 1.2,
    'neighborhood_weight' : 0.5,
    'bias' : 200
}

In [110]:
# Funktion för att uppskatta försäljningspriset på ett hus baserat på antal sovrum osv..och vikter för varje attribut, samt en bias

def estimate_house_sales_price(num_of_bedrooms, sqm, neighborhood, weights):
    price = 0

    price += num_of_bedrooms * weights['bedroom_weight']
    price += sqm * weights['sqm_weight']
    price += neighborhood * weights['neighborhood_weight']
    price += weights['bias']
    return price


In [111]:
# Funktion för att göra förutsägelser baserat på vikter, bias och inputdata

def predict(X, weights, bias):
    return [sum(w*Xj for w, Xj in zip(weights, Xi)) + bias for Xi in X]

# Funktion för gradient_descent-optimering för att anpassa vikter och bias till inputdata och målvariabler
def gradient_descent(X, y, weights, bias, learning_rate, num_iterations):
    # pass
    num_samples, num_features = len(X), len(X[0])

    for _ in range(num_iterations):
        # gör förutsägelser baserat på aktuella vikter och bias
        predictions = predict(X, weights, bias)
        # print(predictions)

        # Beräkna fel mellan förutsägelser och faktiska värden
        errors = [pred - actual for pred, actual in zip(predictions, y)]

        # Beräkna gradient för vikter och bias
        gradient_weights = [sum(err*Xi[i] for err, Xi in zip(errors, X)) / num_samples for i in range(num_features)]
        gradient_bias = sum(errors) / num_samples

        # Uppdatera vikter och bias baserat på gradient och inlärningshastighet
        weights = [w - learning_rate * gw for w, gw in zip(weights, gradient_weights)]
        bias = bias - learning_rate * gradient_bias
    return weights, bias

One Hot Encoding

In [112]:
# Skapa en ny DataFrame "df_encoded"
df_encoded = pd.get_dummies(df, columns = ['neighborhood'])
df_encoded

,bedrooms,sq_meter,sales_price,neighborhood_Cool Town,neighborhood_Normalville,neighborhood_Skid Row
0,3,200,250000,False,True,False
1,2,80,300000,True,False,False
2,2,85,150000,False,True,False
3,1,55,78000,False,True,False
4,4,200,150000,False,False,True


In [113]:
# Visa information om "df_encoded"

df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype
---  ------                    --------------  -----
 0   bedrooms                  5 non-null      int64
 1   sq_meter                  5 non-null      int64
 2   sales_price               5 non-null      int64
 3   neighborhood_Cool Town    5 non-null      bool 
 4   neighborhood_Normalville  5 non-null      bool 
 5   neighborhood_Skid Row     5 non-null      bool 
dtypes: bool(3), int64(3)
memory usage: 267.0 bytes


In [114]:
# Skapa en array "X" som innehåller funktioner genom att ta bort kolumnen "sales_price" från vårt "df_encoded"

X = df_encoded.drop(columns=['sales_price']).values  # Features 
X

array([[3, 200, False, True, False],
       [2, 80, True, False, False],
       [2, 85, False, True, False],
       [1, 55, False, True, False],
       [4, 200, False, False, True]], dtype=object)

In [115]:
# Skapa en array "y" som innehåller målvariabler genom att välja kolumnen "sales_price" från vårt "df_encoded"

y = df_encoded['sales_price'].values  # Target
y

array([250000, 300000, 150000,  78000, 150000], dtype=int64)

In [116]:
# Skapa en lista med vikter baserat på uppgivna vikter i weights_data-dict..
weights =[
    weights_data['bedroom_weight'],
    weights_data['sqm_weight']] + \
    [weights_data['neighborhood_weight']] * 3

weights

[0.8, 1.2, 0.5, 0.5, 0.5]

In [117]:
# [1,2,3] + [4] * 3

[1,2,3] + \
[4] * 3


[1, 2, 3, 4, 4, 4]

In [118]:
# Tilldela värdet av bias från  weights_data-dict till variabeln "bias"

bias = weights_data['bias']
bias

200

In [119]:
# Tilldela värden för inlärnings¨hastighet och antal iterationer

learning_rate = 1e-9
num_iterations = 100

In [126]:
# gradient_descent(X, y, weights, bias, learning_rate, num_iterations)

weights, bias = gradient_descent(X, y, weights, bias, learning_rate, num_iterations)

In [127]:
weights

[0.8464258061476861,
 3.613654212776658,
 0.5059921242312305,
 0.509531582292978,
 0.5029863429868261]

In [124]:
bias

200.01851005136055